In [449]:
import psycopg2
import random
from faker import Faker
fake = Faker(['es_ES'])
from faker import Faker
print("Inicio del proyecto")

Inicio del proyecto


In [450]:
bbdd = 'pkrxcdfv'
usuario = 'pkrxcdfv'
contrasenya = '2Cci5C1lH485Zsv_7_0-TQZOHOf-PasS'
maquina = 'drona.db.elephantsql.com'
puerto = '5432'

conexion = psycopg2.connect(dbname=bbdd,
                            user=usuario,
                            password=contrasenya,
                            host=maquina,
                            port=puerto
)

if conexion.status == 1:
    print("Coneccion con exito")
else:
    print("Error de coneccion")

Coneccion con exito


In [451]:
cursor = conexion.cursor()

In [452]:
cursor.execute(" \
DO $$ BEGIN \
    CREATE TYPE tipo_poliza AS ENUM ('SOAT', 'Poliza_seguro_vehicular', 'SAT', 'TREC', 'RC'); \
EXCEPTION \
    WHEN duplicate_object THEN null; \
END $$;")
conexion.commit()

In [453]:
cursor.execute("CREATE TABLE IF NOT EXISTS POLIZA ( \
      numero_poliza VARCHAR(100) NOT NULL, \
      fecha_inicio DATE NOT NULL, \
      fecha_fin DATE NOT NULL, \
      aseguradora VARCHAR(100) NOT NULL, \
      telefono_aseguradora VARCHAR(30) NOT NULL, \
      telefono_broker VARCHAR(30) NOT NULL, \
      cronograma_pago INT NOT NULL, \
      poliza_adjunta TEXT NOT NULL, \
      tipo_poliza tipo_poliza NOT NULL, \
      estado_poliza BOOLEAN NOT NULL, \
      PRIMARY KEY (numero_poliza) \
    );"
)
conexion.commit()

In [454]:
for i in range (10):
    numero_poliza = fake.bothify(text='???-###-###???', letters='ABCDEFGHY')
    fecha_inicio = fake.date()
    fecha_fin = fake.date()
    aseguradora = fake.company()
    telefono_aseguradora = fake.phone_number()
    telefono_broker = fake.phone_number()
    cronograma_pago = fake.unix_time()
    poliza_adjunta = fake.file_path(depth=5, category='poliza', extension='pdf')
    tipo_poliza =  'SOAT'
    estado_poliza = fake.boolean(chance_of_getting_true=4)
    cursor.execute("INSERT INTO poliza ( \
            numero_poliza, \
            fecha_inicio, \
            fecha_fin, \
            aseguradora, \
            telefono_aseguradora, \
            telefono_broker, \
            cronograma_pago, \
            poliza_adjunta, \
            tipo_poliza, \
            estado_poliza \
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (
        numero_poliza,
        fecha_inicio,
        fecha_fin,
        aseguradora,
        telefono_aseguradora,
        telefono_broker,
        cronograma_pago,
        poliza_adjunta,
        tipo_poliza,
        estado_poliza
    ))
    conexion.commit()
print("Polizas creadas con exito")

Polizas creadas con exito


In [455]:
cursor.execute("SELECT * FROM poliza")
cursor.fetchall()

[('BGB-966-963YEF',
  datetime.date(1990, 5, 8),
  datetime.date(1991, 1, 4),
  'Pelayo PLC',
  '+34 824 584 548',
  '+34879 995 362',
  529407683,
  '/omnis/hic/quibusdam/illum/natus/asperiores.pdf',
  'SOAT',
  False),
 ('YBC-719-639YYY',
  datetime.date(2003, 11, 30),
  datetime.date(2019, 7, 29),
  'Palmer-Cornejo',
  '+34978408514',
  '+34869803206',
  1491827989,
  '/provident/ad/voluptatibus/veritatis/velit/aliquid.pdf',
  'SOAT',
  False),
 ('FFD-054-604YHG',
  datetime.date(1988, 4, 18),
  datetime.date(1981, 9, 21),
  'Valderrama Inc',
  '+34881 558 909',
  '+34807 435 265',
  573105171,
  '/in/unde/quibusdam/nisi/ipsum/molestiae.pdf',
  'SOAT',
  False),
 ('CFE-716-469AGH',
  datetime.date(2006, 8, 8),
  datetime.date(1970, 4, 21),
  'Bosch-Cobo',
  '+34873 22 02 35',
  '+34 860 283 304',
  744654420,
  '/hic/quae/consequatur/autem/dolores/suscipit.pdf',
  'SOAT',
  False),
 ('BFY-994-959DEY',
  datetime.date(2016, 6, 17),
  datetime.date(1976, 1, 11),
  'Calzada PLC',
  '+3

In [456]:
cursor.execute("CREATE TABLE IF NOT EXISTS OBRA ( \
      codigo_obra INT NOT NULL, \
      nombre_obra VARCHAR(100) NOT NULL, \
      direccion VARCHAR(100) NULL, \
      ubicacion VARCHAR(100) NULL, \
      PRIMARY KEY (codigo_obra) \
    );"
)
conexion.commit()

In [457]:
cursor.execute("CREATE TABLE IF NOT EXISTS EMPRESA ( \
      ruc INT NOT NULL, \
      razon_social VARCHAR(100) NOT NULL, \
      direccion VARCHAR(100) NULL, \
      contacto VARCHAR(100) NULL, \
      correo VARCHAR(100) NULL, \
      telefono VARCHAR(30) NOT NULL, \
      PRIMARY KEY (ruc) \
    );"
)
conexion.commit()

In [458]:
cursor.execute("CREATE TABLE IF NOT EXISTS ejecuta ( \
      codigo_obra INT NOT NULL, \
      ruc INT NOT NULL, \
      PRIMARY KEY (codigo_obra, ruc), \
      FOREIGN KEY (codigo_obra) REFERENCES OBRA(codigo_obra), \
      FOREIGN KEY (ruc) REFERENCES EMPRESA(ruc) \
    );"
)
conexion.commit()

In [459]:
for i_empresa in range (10):
    ruc  = fake.unique.random_int(min=111111, max=999999)
    razon_social  = fake.company()
    direccion = fake.address()
    contacto  = fake.name()
    correo  = fake.ascii_company_email()
    telefono  = fake.phone_number()
    cursor.execute("INSERT INTO empresa (ruc, \
            razon_social, \
            direccion, \
            contacto, \
            correo, \
            telefono \
            ) VALUES (%s, %s, %s, %s, %s, %s)", (ruc, razon_social, direccion, contacto, correo, telefono))
    conexion.commit()
    for i_obra in range (2):
        codigo_obra  = fake.unique.random_int(min=111111, max=999999)
        nombre_obra  = fake.company()
        direccion = fake.address()
        ubicacion  = fake.location_on_land()
        cursor.execute("INSERT INTO obra (codigo_obra, \
                nombre_obra, \
                direccion, \
                ubicacion \
                ) VALUES (%s, %s, %s, %s)", (codigo_obra, nombre_obra, direccion, ubicacion))
        conexion.commit()
        cursor.execute("INSERT INTO ejecuta (codigo_obra, ruc ) VALUES (%s, %s)", (codigo_obra, ruc))
        conexion.commit()
print("Registros creados con exito")

Registros creados con exito


In [460]:
conexion.commit()

In [461]:
cursor.execute("SELECT contacto FROM empresa")
empresa = random.choice(cursor.fetchall())[0]
cursor.execute("SELECT ruc FROM empresa WHERE contacto like '%{}%'".format(empresa))
cursor.fetchone()

(658522,)

In [462]:
cursor.execute("SELECT em.ruc, em.razon_social, em.contacto, o.codigo_obra, o.nombre_obra \
    FROM empresa em \
         INNER JOIN ejecuta ej ON ej.ruc = em.ruc \
         INNER JOIN obra o on o.codigo_obra = ej.codigo_obra;")
cursor.fetchall()

[(659255,
  'Agustí-Gil',
  'Rosario Feijoo Folch',
  576921,
  'Vilalta, Gomila and Zabala'),
 (659255, 'Agustí-Gil', 'Rosario Feijoo Folch', 704846, 'Alsina Ltd'),
 (318892,
  'Rocamora, Rey and Riquelme',
  'Ruperta Campillo Clavero',
  389983,
  'Cruz, Company and Morante'),
 (318892,
  'Rocamora, Rey and Riquelme',
  'Ruperta Campillo Clavero',
  272453,
  'Mayol LLC'),
 (236747,
  'Escolano-Rueda',
  'Encarnita Alvarez Castellanos',
  471208,
  'Estévez-Aguirre'),
 (236747,
  'Escolano-Rueda',
  'Encarnita Alvarez Castellanos',
  382102,
  'Álvarez, Tejada and Meléndez'),
 (337562, 'Mena-Cortina', 'Domitila Oliveras Olivera', 932705, 'Arce-Garcia'),
 (337562,
  'Mena-Cortina',
  'Domitila Oliveras Olivera',
  142248,
  'Zurita, Calzada and Ureña'),
 (929725,
  'Aller-Arnaiz',
  'Baltasar Carbonell Pintor',
  243476,
  'Galván, Miralles and Baró'),
 (929725,
  'Aller-Arnaiz',
  'Baltasar Carbonell Pintor',
  910935,
  'Bermúdez-Tormo'),
 (658522,
  'Seguí, Ortuño and Salamanca',
 

In [463]:
cursor.execute("SELECT EM.ruc, COUNT(O.codigo_obra) \
    FROM empresa EM \
         INNER JOIN ejecuta EJ ON EJ.ruc = EM.ruc \
         INNER JOIN obra O on O.codigo_obra = EJ.codigo_obra GROUP BY EM.ruc;")
cursor.fetchall()

[(343473, 2),
 (337562, 2),
 (929725, 2),
 (294661, 2),
 (659255, 2),
 (236747, 2),
 (733937, 2),
 (299069, 2),
 (318892, 2),
 (658522, 2)]

In [464]:
conexion.commit()

In [465]:
cursor.execute("CREATE TABLE IF NOT EXISTS USUARIO ( \
      documento_identidad INT NOT NULL, \
      nombre VARCHAR(100) NULL, \
      email VARCHAR(100) NOT NULL, \
      usuario VARCHAR(100) NOT NULL, \
      contrasenia VARCHAR(100) NOT NULL, \
      fecha_nacimiento DATE NULL, \
      apellidos VARCHAR(100) NULL, \
      ruc INT NOT NULL, \
      PRIMARY KEY (documento_identidad), \
      FOREIGN KEY (ruc) REFERENCES EMPRESA(ruc) \
    );"
)
conexion.commit()

In [466]:
cursor.execute(" \
DO $$ BEGIN \
    CREATE TYPE tipo_vehiculo AS ENUM ('menores', 'pesada', 'carga'); \
EXCEPTION \
    WHEN duplicate_object THEN null; \
END $$;")
conexion.commit()

In [467]:
cursor.execute("CREATE TABLE IF NOT EXISTS ADMINISTRADOR ( \
      documento_identidad INT NOT NULL, \
      PRIMARY KEY (documento_identidad), \
      FOREIGN KEY (documento_identidad) REFERENCES USUARIO(documento_identidad) \
    );"
)
conexion.commit()

In [468]:
for i in range (2):
    cursor.execute("SELECT ruc FROM empresa")
    empresa = random.choice(cursor.fetchall())
    documento_identidad  = fake.unique.random_int(min=111111, max=999999)
    nombre  = fake.name()
    email = fake.ascii_email()
    usuario  = 'administrador'
    contrasenia  = fake.language_name()
    fecha_nacimiento  = fake.date()
    apellidos  = fake.name()
    ruc  = empresa[0]
    cursor.execute("INSERT INTO usuario (documento_identidad, \
            nombre, \
            email, \
            usuario, \
            contrasenia, \
            fecha_nacimiento, \
            apellidos, \
            ruc \
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (documento_identidad, nombre, email, usuario, contrasenia, fecha_nacimiento, apellidos, ruc))
    conexion.commit()
    cursor.execute("INSERT INTO administrador (documento_identidad) VALUES (%s)", [documento_identidad])
    conexion.commit()
print("Usuarios creados con exito")

Usuarios creados con exito


In [469]:
cursor.execute("SELECT U.documento_identidad, U.nombre, U.email, U.usuario \
    FROM usuario U \
         INNER JOIN administrador A ON U.documento_identidad = A.documento_identidad")
cursor.fetchall()

[(913622,
  'Candela Valderrama Barrio',
  'curroroca@alcolea.net',
  'administrador'),
 (885094,
  'Rodolfo del Roman',
  'vilanovainmaculada@morales-mur.es',
  'administrador')]

In [470]:
conexion.commit()

In [471]:
cursor.execute("CREATE TABLE IF NOT EXISTS GESTOR ( \
      documento_identidad INT NOT NULL, \
      PRIMARY KEY (documento_identidad), \
      FOREIGN KEY (documento_identidad) REFERENCES USUARIO(documento_identidad) \
    );"
)
conexion.commit()

In [472]:
for i in range (2):
    cursor.execute("SELECT ruc FROM empresa")
    empresa = random.choice(cursor.fetchall())
    documento_identidad  = fake.unique.random_int(min=111111, max=999999)
    nombre  = fake.name()
    email = fake.ascii_email()
    usuario  = 'gestor'
    contrasenia  = fake.language_name()
    fecha_nacimiento  = fake.date()
    apellidos  = fake.name()
    ruc  = empresa[0]
    cursor.execute("INSERT INTO usuario (documento_identidad, \
            nombre, \
            email, \
            usuario, \
            contrasenia, \
            fecha_nacimiento, \
            apellidos, \
            ruc \
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (documento_identidad, nombre, email, usuario, contrasenia, fecha_nacimiento, apellidos, ruc))
    conexion.commit()
    cursor.execute("INSERT INTO gestor (documento_identidad) VALUES (%s)", [documento_identidad])
    conexion.commit()
print("Usuarios creados con exito")

Usuarios creados con exito


In [473]:
cursor.execute("SELECT U.documento_identidad, U.nombre, U.email, U.usuario \
    FROM usuario U \
         INNER JOIN gestor A ON U.documento_identidad = A.documento_identidad")
cursor.fetchall()

[(934479, 'Marcela Huguet Daza', 'ssastre@carro.com', 'gestor'),
 (713191, 'Sosimo Cordero', 'borjasantiago@gmail.com', 'gestor')]

In [474]:
cursor.execute("CREATE TABLE IF NOT EXISTS CONDUCTOR ( \
      documento_identidad INT NOT NULL, \
      PRIMARY KEY (documento_identidad), \
      FOREIGN KEY (documento_identidad) REFERENCES USUARIO(documento_identidad) \
    );"
)
conexion.commit()

In [475]:
for i in range (2):
    cursor.execute("SELECT ruc FROM empresa")
    empresa = random.choice(cursor.fetchall())
    documento_identidad  = fake.unique.random_int(min=111111, max=999999)
    nombre  = fake.name()
    email = fake.ascii_email()
    usuario  = 'conductor'
    contrasenia  = fake.language_name()
    fecha_nacimiento  = fake.date()
    apellidos  = fake.name()
    ruc  = empresa[0]
    cursor.execute("INSERT INTO usuario (documento_identidad, \
            nombre, \
            email, \
            usuario, \
            contrasenia, \
            fecha_nacimiento, \
            apellidos, \
            ruc \
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (documento_identidad, nombre, email, usuario, contrasenia, fecha_nacimiento, apellidos, ruc))
    conexion.commit()
    cursor.execute("INSERT INTO conductor (documento_identidad) VALUES (%s)", [documento_identidad])
    conexion.commit()
print("Usuarios creados con exito")

Usuarios creados con exito


In [476]:
cursor.execute("SELECT U.documento_identidad, U.nombre, U.email, U.usuario \
    FROM usuario U \
         INNER JOIN conductor A ON U.documento_identidad = A.documento_identidad")
cursor.fetchall()

[(688791, 'Cebrián Barón Figueroa', 'climaco49@yahoo.com', 'conductor'),
 (181868, 'Ismael Amador Batlle', 'gsevilla@hotmail.com', 'conductor')]

In [477]:
cursor.execute("CREATE TABLE IF NOT EXISTS VEHICULO ( \
      placa VARCHAR(10) NOT NULL, \
      numero_bastidor VARCHAR(30) NOT NULL, \
      tipo_vehiculo tipo_vehiculo NOT NULL, \
      fotografia_vehiculo INT NOT NULL, \
      ruc INT NOT NULL, \
      PRIMARY KEY (placa), \
      FOREIGN KEY (ruc) REFERENCES EMPRESA(ruc), \
      UNIQUE (numero_bastidor) \
    );"
)
conexion.commit()

In [478]:
cursor.execute("CREATE TABLE IF NOT EXISTS MENOR ( \
      auto BOOLEAN NOT NULL DEFAULT FALSE, \
      suv BOOLEAN NOT NULL DEFAULT FALSE, \
      placa VARCHAR(10) NOT NULL, \
      PRIMARY KEY (placa), \
      FOREIGN KEY (placa) REFERENCES VEHICULO(placa) \
    );"
)
conexion.commit()

In [479]:
cursor.execute("CREATE TABLE IF NOT EXISTS PESADO ( \
      minicargador BOOLEAN NOT NULL DEFAULT FALSE, \
      cargador BOOLEAN NOT NULL DEFAULT FALSE, \
      retroexcavadora BOOLEAN NOT NULL DEFAULT FALSE, \
      excavadora BOOLEAN NOT NULL DEFAULT FALSE, \
      rodillo BOOLEAN NOT NULL DEFAULT FALSE, \
      placa VARCHAR(10) NOT NULL, \
      PRIMARY KEY (placa), \
      FOREIGN KEY (placa) REFERENCES VEHICULO(placa) \
    );"
)
conexion.commit()

In [480]:
cursor.execute("CREATE TABLE IF NOT EXISTS CARGA ( \
      volquete BOOLEAN NOT NULL DEFAULT FALSE, \
      placa VARCHAR(10) NOT NULL, \
      PRIMARY KEY (placa), \
      FOREIGN KEY (placa) REFERENCES VEHICULO(placa) \
    );"
)
conexion.commit()

In [481]:
cursor.execute("CREATE TABLE IF NOT EXISTS tiene ( \
      placa VARCHAR(10) NOT NULL, \
      numero_poliza VARCHAR(100) NOT NULL, \
      PRIMARY KEY (placa, numero_poliza), \
      FOREIGN KEY (placa) REFERENCES VEHICULO(placa), \
      FOREIGN KEY (numero_poliza) REFERENCES POLIZA(numero_poliza) \
    );"
)
conexion.commit()

In [482]:
cursor.execute("CREATE TABLE IF NOT EXISTS asignado ( \
      documento_identidad INT NOT NULL, \
      placa VARCHAR(10) NOT NULL, \
      PRIMARY KEY (documento_identidad, placa), \
      FOREIGN KEY (documento_identidad) REFERENCES CONDUCTOR(documento_identidad), \
      FOREIGN KEY (placa) REFERENCES VEHICULO(placa) \
    );"
)
conexion.commit()

In [483]:
cursor.execute("CREATE TABLE IF NOT EXISTS gestiona ( \
      codigo_obra INT NOT NULL, \
      documento_identidad INT NOT NULL, \
      PRIMARY KEY (codigo_obra, documento_identidad), \
      FOREIGN KEY (codigo_obra) REFERENCES OBRA(codigo_obra), \
      FOREIGN KEY (documento_identidad) REFERENCES GESTOR(documento_identidad) \
    );"
)
conexion.commit()

In [484]:
cursor.execute("ALTER TABLE vehiculo ALTER COLUMN fotografia_vehiculo TYPE text;")
conexion.commit()

In [495]:
conexion.commit()

In [496]:
for i in range (10):
    cursor.execute("SELECT ruc FROM empresa")
    empresa = random.choice(cursor.fetchall())[0]
    placa  = fake.bothify(text='?####??', letters='ABCDE')
    numero_bastidor  = fake.bothify(text='#-??###-###???', letters='ABCDEFGHY')
    tipo_vehiculo = 'menores'
    fotografia_vehiculo = fake.image(size=(16, 16), hue=[90, 270], image_format='ico')
    ruc = empresa
    cursor.execute("INSERT INTO vehiculo ( \
            placa, \
            numero_bastidor, \
            tipo_vehiculo, \
            fotografia_vehiculo, \
            ruc \
            ) VALUES (%s, %s, %s, %s, %s)", (placa, numero_bastidor, tipo_vehiculo, fotografia_vehiculo, ruc))
    conexion.commit()
print("Vehiculos creados creados con exito")

Vehiculos creados creados con exito


In [506]:
cursor.execute("SELECT placa FROM vehiculo WHERE placa like '%C' OR placa like '%A'")
cursor.fetchall()

[('E3276EC',), ('A1834CC',), ('E2353DA',)]

In [502]:
conexion.commit()

In [ ]:
print(fake.bothify(text='??######???', letters='ABCDE'))

In [ ]:
conexion.close()